In [7]:
import numpy as np
import os
import PIL
from PIL import Image

from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [8]:
batch_size = 32

# This is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


In [9]:
os.getcwd()

'D:\\INSOFE\\CSE 7124c Text Mining\\CUTe'

In [10]:
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data

train_generator = train_datagen.flow_from_directory(
        'D:\\INSOFE\\CSE 7124c Text Mining\\CUTe\\10_categories\\train',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 224*224
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels


Found 2614 images belonging to 10 classes.


In [11]:
# This is the augmentation configuration we will use for testing:
# Only rescaling. Other transformations are not required for testing. Duh!
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
# validation set

validation_generator = test_datagen.flow_from_directory(
        'D:\\INSOFE\\CSE 7124c Text Mining\\CUTe\\10_categories\\test',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 224*224
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

Found 1110 images belonging to 10 classes.


In [13]:
print(train_generator.class_indices)

{'BACKGROUND_Google': 0, 'Faces': 1, 'Faces_easy': 2, 'Leopards': 3, 'Motorbikes': 4, 'airplanes': 5, 'bonsai': 6, 'car_side': 7, 'grand_piano': 8, 'watch': 9}


In [14]:
# Check the corresponding filenames of the predictions
print(train_generator.filenames[:5])

['BACKGROUND_Google\\image_0101.jpg', 'BACKGROUND_Google\\image_0102.jpg', 'BACKGROUND_Google\\image_0103.jpg', 'BACKGROUND_Google\\image_0104.jpg', 'BACKGROUND_Google\\image_0105.jpg']


# Model 1-  Sequential Model

In [15]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [16]:
# fit_generator is similar to 'fit'. But instead of x_train and y_train, we pass 'train_generator' 
# which already has the samples and their corresponding target information.

# 'step' here is one mini-batch
# 'steps_per_epoch' is number of batches per epoch.
# Typically 'steps_per_epoch' should be total_samples divided by batch_size
model.fit_generator(
        train_generator,
        steps_per_epoch=120//batch_size, # '//' in python returns only the quotient
        epochs=20,
        validation_data=validation_generator,
        validation_steps=45//batch_size)

model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/20
3/3 [==============================] - 27s 9s/step - loss: 3.6125 - acc: 0.1250 - val_loss: 2.2415 - val_acc: 0.2500
Epoch 2/20
3/3 [==============================] - 14s 5s/step - loss: 2.4090 - acc: 0.2604 - val_loss: 2.1741 - val_acc: 0.1562
Epoch 3/20
3/3 [==============================] - 14s 5s/step - loss: 2.1464 - acc: 0.2292 - val_loss: 2.0916 - val_acc: 0.2188
Epoch 4/20
3/3 [==============================] - 14s 5s/step - loss: 2.1446 - acc: 0.2292 - val_loss: 2.1822 - val_acc: 0.1875
Epoch 5/20
3/3 [==============================] - 14s 5s/step - loss: 2.0980 - acc: 0.2188 - val_loss: 2.0590 - val_acc: 0.3750
Epoch 6/20
3/3 [==============================] - 13s 4s/step - loss: 2.1113 - acc: 0.1875 - val_loss: 1.9866 - val_acc: 0.3750
Epoch 7/20
3/3 [==============================] - 13s 4s/step - loss: 2.0207 - acc: 0.2917 - val_loss: 2.1295 - val_acc: 0.2812
Epoch 8/20
3/3 [==============================] - 13s 4s/step - loss: 1.8824 - acc: 0.4167 - val_loss: 2

In [17]:
test_generator = test_datagen.flow_from_directory( 
        'D:\\INSOFE\\CSE 7124c Text Mining\\CUTe\\10_categories\\final_test',
        target_size=(224,224), batch_size=batch_size,class_mode='categorical')

Found 32 images belonging to 1 classes.


In [18]:
test_prob = model.predict_generator(test_generator, steps=2) # this returns the probabilities
test_pred_classes = np.argmax(test_prob, axis=1) # convert probabilities to classes
print(test_pred_classes)

[4 4 0 4 4 0 0 4 0 4 4 0 0 4 4 0 4 4 4 0 0 4 0 4 4 0 4 4 4 4 4 0 4 0 4 4 4
 4 0 0 0 0 4 4 4 0 4 4 4 0 0 4 0 4 4 4 0 4 4 0 4 4 0 4]


In [48]:
print(train_generator.class_indices)

{'BACKGROUND_Google': 0, 'Faces': 1, 'Faces_easy': 2, 'Leopards': 3, 'Motorbikes': 4, 'airplanes': 5, 'bonsai': 6, 'car_side': 7, 'grand_piano': 8, 'watch': 9}


In [50]:
# Check the corresponding filenames of the predictions
print(test_generator.filenames[:18])

['final_test\\image_0011.jpg', 'final_test\\image_0012.jpg', 'final_test\\image_0013.jpg', 'final_test\\image_0014.jpg', 'final_test\\image_0016.jpg', 'final_test\\image_0017.jpg', 'final_test\\image_0018.jpg', 'final_test\\image_0019.jpg', 'final_test\\image_0020.jpg', 'final_test\\image_0021.jpg', 'final_test\\image_0022.jpg', 'final_test\\image_0037.jpg', 'final_test\\image_0038.jpg', 'final_test\\image_0039.jpg', 'final_test\\image_0129.jpg', 'final_test\\image_0130.jpg', 'final_test\\image_0131.jpg', 'final_test\\image_0132.jpg']


# Model 2 - VGG Model

In [1]:
# Get VGG16 architecture from keras.applications
from keras.applications.inception_v3 import InceptionV3, decode_predictions

C:\Users\kalya\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.layers import GlobalAveragePooling2D

In [19]:
trained_model = InceptionV3(include_top=False,weights='imagenet')

87916544/87910968 [==============================] - 102s 1us/step


In [26]:
x = trained_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(300,activation='relu')(x)
x = BatchNormalization()(x)
pred_inception= Dense(10,activation='softmax')(x)
model1 = Model(inputs=trained_model.input,outputs=pred_inception)

In [27]:
print(model1.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_4[0][0]                   
__________________________________________________________________________________________________
activation_6 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_5 (

In [28]:
#making the layers of vgg16 non-trainable
for layer in trained_model.layers:
    layer.trainable=False
    
#compiling the model
adam = Adam(lr=0.001)
model1.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=adam)

In [29]:
model1.fit_generator(
        train_generator,
        steps_per_epoch=120//batch_size, # '//' in python returns only the quotient
        epochs=20,
        validation_data=validation_generator,
        validation_steps=45//batch_size)

model1.save_weights('first_try.h5')

Epoch 1/20
3/3 [==============================] - 88s 29s/step - loss: 1.8559 - acc: 0.3677 - val_loss: 0.7818 - val_acc: 0.7188
Epoch 2/20
3/3 [==============================] - 85s 28s/step - loss: 0.7043 - acc: 0.7708 - val_loss: 1.3720 - val_acc: 0.7500
Epoch 3/20
3/3 [==============================] - 84s 28s/step - loss: 0.6460 - acc: 0.7812 - val_loss: 1.2369 - val_acc: 0.7812
Epoch 4/20
3/3 [==============================] - 84s 28s/step - loss: 0.3561 - acc: 0.8646 - val_loss: 0.3921 - val_acc: 0.8750
Epoch 5/20
3/3 [==============================] - 84s 28s/step - loss: 0.4171 - acc: 0.8438 - val_loss: 0.2055 - val_acc: 0.9062
Epoch 6/20
3/3 [==============================] - 87s 29s/step - loss: 0.3844 - acc: 0.8333 - val_loss: 0.5234 - val_acc: 0.8125
Epoch 7/20
3/3 [==============================] - 90s 30s/step - loss: 0.3962 - acc: 0.8542 - val_loss: 0.5865 - val_acc: 0.8750
Epoch 8/20
3/3 [==============================] - 88s 29s/step - loss: 0.3705 - acc: 0.8958 - val

In [30]:
test_prob = model1.predict_generator(test_generator, steps=2) # this returns the probabilities
test_pred_classes = np.argmax(test_prob, axis=1) # convert probabilities to classes
print(test_pred_classes)

[4 4 0 0 7 4 6 0 7 4 0 8 7 4 8 4 7 7 0 4 4 7 8 6 7 0 4 4 6 6 4 4 4 7 4 4 7
 7 8 4 4 0 7 6 6 4 0 4 7 7 8 0 4 0 4 7 6 0 4 0 4 6 4 8]


In [31]:
print(train_generator.class_indices)

{'BACKGROUND_Google': 0, 'Faces': 1, 'Faces_easy': 2, 'Leopards': 3, 'Motorbikes': 4, 'airplanes': 5, 'bonsai': 6, 'car_side': 7, 'grand_piano': 8, 'watch': 9}


In [32]:
print(test_generator.filenames[:18])

['final_test\\image_0011.jpg', 'final_test\\image_0012.jpg', 'final_test\\image_0013.jpg', 'final_test\\image_0014.jpg', 'final_test\\image_0016.jpg', 'final_test\\image_0017.jpg', 'final_test\\image_0018.jpg', 'final_test\\image_0019.jpg', 'final_test\\image_0020.jpg', 'final_test\\image_0021.jpg', 'final_test\\image_0022.jpg', 'final_test\\image_0037.jpg', 'final_test\\image_0038.jpg', 'final_test\\image_0039.jpg', 'final_test\\image_0129.jpg', 'final_test\\image_0130.jpg', 'final_test\\image_0131.jpg', 'final_test\\image_0132.jpg']
